In [1]:
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd 
import os
import seaborn as sns
from pylab import savefig
import matplotlib

In [2]:
# Create new folder in root path 
def createFolder(rootPath, folderName): 
    folderPath = os.path.join(rootPath, folderName) 
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    return folderPath + "\\" 

In [12]:
def windRosePlot(df, figurePath): 
    ws = df['Direction'].values
    wd = df['Distance'].values
    ax = WindroseAxes.from_ax()
    ax.contourf(ws, wd, cmap=cm.hot)
    ax.set_legend()
    ax.figure.savefig(figurePath + fr + "_" + str(yr) + '.png', dpi = 400)
    ax.figure.clf()

In [4]:
def shpFiles(rootPath, ext):
    emptyList = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                emptyList.append(os.path.join(path, names))
    return(emptyList)

In [5]:
# Weighted Average DEF 
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [6]:
# find quartile 
def quartiles(dataPoints):
    # check the input is not empty
    # 1. order the data set
    sortedPoints = sorted(dataPoints)
    # 2. divide the data set in two halves
    mid = len(sortedPoints) // 2 # uses the floor division to have integer returned
    if (len(sortedPoints) % 2 == 0):
        # even
        lowerQ = np.median(sortedPoints[:mid])
        upperQ = np.median(sortedPoints[mid:])
    else:
        # odd
        lowerQ = np.median(sortedPoints[:mid])  # same as even
        upperQ = np.median(sortedPoints[mid+1:])
    return (lowerQ, upperQ)

In [7]:
FinalDF = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\FinalDF.csv', index_col=0)
FinalDF = FinalDF[~FinalDF['Fire'].isin(['Colby', 'Cherokee', 'Falls'])]
sample = FinalDF.groupby(['Fire', 'Year']).size().reset_index()
fire = sample['Fire'].tolist()
FIRES = sample['Fire'].tolist()
YEARS =  sample['Year'].tolist()

In [13]:
sns.set(style="whitegrid")
palette = sns.color_palette("mako_r", 3)
rootPath = r"F:\DriversFireProject\MultiSpread" 
for fr, yr in zip(FIRES[220:], YEARS[220:]): 
    print(fr, yr)
    flname = fr + "_" + str(yr) + "_FinalAll.csv"
    CSVPath = os.path.join(rootPath, str(yr), fr, "FinalCSV", flname)
    figurePath = createFolder(os.path.join(rootPath, str(yr), fr), "Figures")
    df = pd.read_csv(CSVPath, index_col = 0) 
    windRosePlot(df, figurePath)
    frame1 = df.groupby(["JulianDay"]).apply(wavg, "Direction", "Distance").reset_index()
    frame1.columns = ['JulianDay', '(Direction, WA)']
    f = {'Area' : ['median'], 'Distance': ['max', 'median', 'std']}
    frame2 = df.groupby("JulianDay").agg(f).reset_index()
    frame2.columns = ['JulianDay', 'Area (ha)', 'Magnitude (max)','DIST_Median','DIST_STD']
    frame3 = df.loc[df.reset_index().groupby(['JulianDay'])['Distance'].idxmax()].reset_index()
    frame3 = frame3[['JulianDay', 'Direction']]
    frame3.columns = ['JulianDay', '(Direction, @MaxDist)']
    finalDF = frame1.merge(frame2, left_on='JulianDay', right_on='JulianDay')
    finalDF = finalDF.merge(frame3, left_on='JulianDay', right_on='JulianDay').round(3)
    cols = ['JulianDay','DIR_WA', 'Area (ha)', 'Magnitude (max)','DIST_Median','DIST_STD', 'DIR_MaxDist']
    finalDF.columns = cols
    finalDF['DIST_Median'] = finalDF['DIST_Median'].astype(str)
    finalDF['DIST_STD'] = finalDF['DIST_STD'].astype(str)
    finalDF['Magnitude (median)'] = finalDF['DIST_Median']
    finalDF['Magnitude (stdv)'] = finalDF['DIST_STD']
    finalDF['DIST_MEDSTD'] = finalDF[['DIST_Median', 'DIST_STD']].apply(lambda x: ' ± '.join(x[x.notnull()]), axis = 1)
    cols = ['JulianDay','Area (ha)', 'DIR_WA', 'DIR_MaxDist', 'DIST_MEDSTD', 'Magnitude (max)', 'Magnitude (median)', 'Magnitude (stdv)']
    finalDF = finalDF[cols]
    lowerQuart = df['Distance'].groupby(df["JulianDay"], group_keys=False).apply(lambda x: quartiles(x)[0]).reset_index()
    upperQuart = df['Distance'].groupby(df["JulianDay"], group_keys=False).apply(lambda x: quartiles(x)[1]).reset_index()
    quart = lowerQuart.merge(upperQuart, left_on='JulianDay', right_on='JulianDay')
    quart.columns = ['JulianDay', 'Magnitude (lowerQ)', 'Magnitude (upperQ)']
    finalDF = finalDF.merge(quart, left_on='JulianDay', right_on='JulianDay').round(3)
    finalDF['Name'] = fr
    csvName = fr + "_" + str(yr) + "_summary.csv"
    csvPath = createFolder(rootPath, "SummaryCSV")
    yrPath = createFolder(csvPath, str(yr))
    finalDF.to_csv(os.path.join(yrPath, csvName))
    # Plot the responses for different events and regions 
    matplotlib.pyplot.close('all')
    finalDF['log(Area)'] = np.log(finalDF['Area (ha)'])
    melted = pd.melt(finalDF, id_vars=['JulianDay'], value_vars=['log(Area)', 'Magnitude (max)', 'Magnitude (median)'],
        var_name='Value Type', value_name='Values')
    melted['Values'] = melted['Values'].astype('float64')
    snsPLOT = sns.lineplot(x="JulianDay", y="Values", hue = 'Value Type', style = 'Value Type', palette = palette, data=melted)
    figure = snsPLOT.get_figure()    
    figure.savefig(figurePath + fr + "_" + str(yr) + "_timeseries.png", dpi = 400)
    matplotlib.pyplot.close('all')
    matplotlib.pyplot.clf()
    
    


South 2019
Southcomplexcastle 2015
Southcomplexpattison 2015
Southcomplexpelletreau 2015
Southfork 2017
Springs 2019
Steele 2017
Stonemdf 2018
Sulfur 2017
Summitcomplexmccormick 2017
Swedes 2013
Tassajarra 2015
Tesla 2015
Thomas 2018
Tomahawk 2014
Trailhead 2016
Tubbs 2017
Tucker 2019
Turkey 2012
Valley 2018
Walker 2019
Wall 2017
Way 2014
Whaleback 2018
White 2013
Whittier 2017
Willard 2016
Williams 2012
Willow 2015
Winter 2017
Woolsey 2018
Wragg 2015
Wye 2012
Young 2017


<Figure size 432x288 with 0 Axes>